# Overlord AI
For information about this project, see [this README.md](../doc/README.md)


## TODO:

* [x] notebook source list
* [x] notebook downloads
* [ ] dataset downloads
* [ ] dataset processing
* [ ] model training

# CODE STARTS HERE

First we import all the libraries and whatnot

# imports


In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
import re

# definitions

I added an optional download for the used workbooks so you have them saved locally for when you want to use them. When set to True, all the models will be downloaded to ../data/.models/

In [3]:
downloadWorkbooks = False

# reading data


## data gathering

In [4]:
df = pd.DataFrame([], columns=[
                  'index', 'link', 'requeststatus', 'content'])
index = 0
sourcelist = pd.read_csv("../data/models/sourcesShort.csv")
# print(sourcelist)
for item in sourcelist['Sources']:  # loop through sources
    r = requests.get(item)  # get the html data

    if (downloadWorkbooks):
        # download the workbooks
        kagglelink = item.replace('https://www.kaggle.com/code/', "")
        dirname = kagglelink.split("/")[1]
        command = "kaggle k pull " + kagglelink + " -p ../data/.models/"+dirname
        os.system('%s' % command)

# add data to the df
    new_row = [index, item, r.status_code, r.text]
    df.loc[index] = new_row
    index += 1
print("data gathered")

data gathered


## transform data

### get execution time

In [5]:
# html parsing
# grep -ioe "runtimeseconds\"\:[0-9]*\.[0-9]*" # grep to extract the run time form html
for index, row in df.iterrows():
    print(index)
    # print(row['content'])
    runTime = float(re.split(":", re.findall(
        "\"runTimeSeconds\"\:[0-9]+\.[0-9]+", row['content'])[0])[1])
    print("runtime in seconds: %f"%runTime)
    df.loc[df['index'] == index, 'runTime'] = runTime


0
runtime in seconds: 861.112981
1
runtime in seconds: 1206.104310
2
runtime in seconds: 20.786203
3
runtime in seconds: 386.320938
4
runtime in seconds: 22.583234
5
runtime in seconds: 48.534224
6
runtime in seconds: 49.849152
7
runtime in seconds: 2790.507509
8
runtime in seconds: 5754.205438
9
runtime in seconds: 10.160655
10
runtime in seconds: 3534.495725
11
runtime in seconds: 1225.858824
12
runtime in seconds: 8293.278067
13
runtime in seconds: 392.470415
14
runtime in seconds: 13.538386
15
runtime in seconds: 2393.242585
16
runtime in seconds: 1938.983198
17
runtime in seconds: 27493.298228
18
runtime in seconds: 420.714699
19
runtime in seconds: 2261.530650


### get upvotes

In [6]:
# html parsing
# grep -ioe "runtimeseconds\"\:[0-9]*\.[0-9]*" # grep to extract the run time form html
for index, row in df.iterrows():
    print(index)
    # print(row['content'])
    upvoteCount = int(re.split(":", re.findall(
        "\"upvoteCount\"\:[0-9]+", row['content'])[0])[1])
    print("upvotes: %d"%upvoteCount)
    df.loc[df['index'] == index, 'upvoteCount'] = upvoteCount


0
upvotes: 4192
1
upvotes: 3346
2
upvotes: 2688
3
upvotes: 2496
4
upvotes: 2303
5
upvotes: 2255
6
upvotes: 1657
7
upvotes: 1250
8
upvotes: 1209
9
upvotes: 1117
10
upvotes: 1092
11
upvotes: 951
12
upvotes: 888
13
upvotes: 872
14
upvotes: 767
15
upvotes: 610
16
upvotes: 580
17
upvotes: 564
18
upvotes: 547
19
upvotes: 526


### get tags

In [7]:
for index, row in df.iterrows():
    print(index)
    # "tags"":[]
    tags = re.findall("\"tags\"\:\[.+\"\}\]\,\"upvoteCount\"", row['content'])
    # print(tags)
    tagsFormatted = ""
    df.loc[df['index'] == index, 'tags'] = tagsFormatted # this is here so it's before all the hot encoded stuff
    if (tags):
        for iter in range(40):
            temp = re.findall("\"slug\"\:\".{%d}\"\,\"url" % iter, tags[0])

            if (temp):
                temp = re.split(':|,|"', temp[0])[4]
                #print(temp)
                # hot encoded tags
                df.loc[df['index'] == index, temp] = 1
                # fill the empty values
                df[temp] = df[temp].fillna(0)
                tagsFormatted = tagsFormatted+ temp + " "
        
    print(tagsFormatted) # human readable, space delimited list of the tags
    df.loc[df['index'] == index, 'tags'] = tagsFormatted


0
finance classification data-visualization dimensionality-reduction 
1
eda beginner classification 
2
classification feature-engineering 
3
eda ensembling classification model-comparison data-visualization 
4
beginner classification 
5
beginner deep-learning classification neural-networks 
6
healthcare classification data-visualization feature-engineering 
7
eda beginner classification 
8
eda classification feature-engineering 
9
eda beginner classification data-visualization feature-engineering 
10
eda business classification data-visualization feature-engineering 
11
marketing clustering classification 
12
cnn beginner classification image-processing binary-classification 
13
eda finance beginner classification data-visualization 
14
numpy pandas seaborn beginner matplotlib deep-learning classification health-conditions logistic-regression binary-classification 
15
eda classification data-visualization feature-engineering 
16
nlp beginner classification 
17
cnn deep-learning classif

## export data

In [8]:
df.to_csv('kaggleScrape.csv', index=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 0 to 19
Data columns (total 34 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   index                      20 non-null     int64  
 1   link                       20 non-null     object 
 2   requeststatus              20 non-null     int64  
 3   content                    20 non-null     object 
 4   runTime                    20 non-null     float64
 5   upvoteCount                20 non-null     float64
 6   tags                       20 non-null     object 
 7   finance                    20 non-null     float64
 8   classification             20 non-null     float64
 9   data-visualization         20 non-null     float64
 10  dimensionality-reduction   20 non-null     float64
 11  eda                        20 non-null     float64
 12  beginner                   20 non-null     float64
 13  feature-engineering        20 non-null     float64
 

In [10]:
df.describe()

,index,requeststatus,runTime,upvoteCount,finance,classification,data-visualization,dimensionality-reduction,eda,beginner,...,binary-classification,numpy,pandas,seaborn,matplotlib,health-conditions,logistic-regression,nlp,multiclass-classification,tabular-data
count,20.00000,20.0,20.000000,20.000000,20.000000,20.0,20.000000,20.000000,20.000000,20.000000,...,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,9.50000,200.0,2955.878771,1495.500000,0.100000,1.0,0.400000,0.050000,0.500000,0.500000,...,0.100000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000
std,5.91608,0.0,6164.811268,1044.697161,0.307794,0.0,0.502625,0.223607,0.512989,0.512989,...,0.307794,0.223607,0.223607,0.223607,0.223607,0.223607,0.223607,0.223607,0.223607,0.223607
min,0.00000,200.0,10.160655,526.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.75000,200.0,49.520420,727.750000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,9.50000,200.0,1033.608645,1104.500000,0.000000,1.0,0.000000,0.000000,0.500000,0.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,14.25000,200.0,2492.558816,2267.000000,0.000000,1.0,1.000000,0.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,19.00000,200.0,27493.298228,4192.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
df.sample(5)

,index,link,requeststatus,content,runTime,upvoteCount,tags,finance,classification,data-visualization,...,binary-classification,numpy,pandas,seaborn,matplotlib,health-conditions,logistic-regression,nlp,multiclass-classification,tabular-data
16,16,https://www.kaggle.com/code/sbongo/for-beginne...,200,"\r\n\r\n<!DOCTYPE html>\r\n<html lang=""en"">\r\...",1938.983198,580.0,nlp beginner classification,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,9,https://www.kaggle.com/code/vbmokin/data-scien...,200,"\r\n\r\n<!DOCTYPE html>\r\n<html lang=""en"">\r\...",10.160655,1117.0,eda beginner classification data-visualization...,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,12,https://www.kaggle.com/code/uysimty/keras-cnn-...,200,"\r\n\r\n<!DOCTYPE html>\r\n<html lang=""en"">\r\...",8293.278067,888.0,cnn beginner classification image-processing b...,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,19,https://www.kaggle.com/code/vbmokin/nlp-eda-ba...,200,"\r\n\r\n<!DOCTYPE html>\r\n<html lang=""en"">\r\...",2261.530650,526.0,eda deep-learning classification data-visualiz...,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,14,https://www.kaggle.com/code/cdabakoglu/heart-d...,200,"\r\n\r\n<!DOCTYPE html>\r\n<html lang=""en"">\r\...",13.538386,767.0,numpy pandas seaborn beginner matplotlib deep-...,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
